In [1]:
import numpy as np
import pandas as pd
import random
import torch

In [2]:
# Set the random seed for reproducibility
RANDOM_STATE = 0
N_JOBS = 8
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

In [3]:
HYPERPARAMETERS = {
    'Input Preprocessing' : {
        'Mask Proportions' : [0.1, 0.2, 0.4, 0.8],
        'Batch Size' : 4
    },
    'Input Embedding' : {
        'Surface Embedding' : {
            'Grid Dimension' : 3,
            'Channels Dimension' : 8,
        },
        'Pre-Encoder' : {
            'Branch Channels Dimension' : 4,
            'Number of Blocks' : 2,
        }
    },
    'Surface Encoding' : {
        'Encoder' : {
            'Number of Heads' : 4,
            'Hidden Dimension' : 16,
            'Dropout' : 0.1,
            'Number of Blocks' : 2,
            'External Feature Dimension' : 3,
        }
    },
    'Query Embedding' : {
        'Pre-Decoder' : {
            'Hidden Dimension' : 16,
            'Dropout' : 0.1,
            'Number of Blocks' : 2,
        }
    },
    'Surface Decoding' : {
        'Decoder' : {
            'Number of Heads' : 4,
            'Hidden Dimension' : 16,
            'Dropout' : 0.1,
            'Number of Blocks' : 2,
        }
    },
    'No-Arbitrage' : {
        'Butterfly' : 1,
        'Calendar' : 1,
    }
}

## Dataset

In [4]:
aapl_googl_data = pd.read_csv('volatility_surface_AAPL_GOOGL_2013_01_2013_06.csv', parse_dates=True, index_col=[0, 1], date_format="ISO8601")
aapl_googl_data

Log Moneyness  Time to Maturity  Implied Volatility  \
Datetime   Symbol                                                        
2013-01-02 AAPL        -0.316688          0.007937              0.3726   
           AAPL        -0.316688          0.007937              0.6095   
           AAPL        -0.304266          0.007937              0.3726   
           AAPL        -0.304266          0.007937              0.6095   
           AAPL        -0.291996          0.007937              0.3726   
...                          ...               ...                 ...   
2013-06-28 GOOGL        0.427518          2.253968              0.2430   
           GOOGL        0.434898          2.253968              0.2383   
           GOOGL        0.434898          2.253968              0.2426   
           GOOGL        0.442224          2.253968              0.2402   
           GOOGL        0.442224          2.253968              0.2433   

                   Market Return  Market Volatility  Treasury Rate  
Datetime   Symbol                                                   
2013-01-02 AAPL         0.025086          14.680000          0.055  
           AAPL         0.025086          14.680000          0.055  
           AAPL         0.025086          14.680000          0.055  
           AAPL         0.025086          14.680000          0.055  
           AAPL         0.025086          14.680000          0.055  
...                          ...                ...            ...  
2013-06-28 GOOGL       -0.004299          16.860001          0.030  
           GOOGL       -0.004299          16.860001          0.030  
           GOOGL       -0.004299          16.860001          0.030  
           GOOGL       -0.004299          16.860001          0.030  
           GOOGL       -0.004299          16.860001          0.030  

[574326 rows x 6 columns]

In [5]:
def implied_volatility_surfaces(options_market_data):
    # Group the data by Datetime and Symbol
    grouped_data = options_market_data.groupby(level=['Datetime', 'Symbol'])

    surfaces = []
    for (date, symbol), surface in grouped_data:
        surface_dict = {
            'Datetime': date,
            'Symbol': symbol,
            'Market Features': {
                'Market Return': surface['Market Return'].values[0],
                'Market Volatility': surface['Market Volatility'].values[0],
                'Treasury Rate': surface['Treasury Rate'].values[0],
            },
            'Surface': {
                'Log Moneyness': surface['Log Moneyness'].values,
                'Time to Maturity': surface['Time to Maturity'].values,
                'Implied Volatility': surface['Implied Volatility'].values,
            }
        }
        surfaces.append(surface_dict)

    return surfaces

surfaces = implied_volatility_surfaces(aapl_googl_data)
surfaces[0]

{'Datetime': Timestamp('2013-01-02 00:00:00'),
 'Symbol': 'AAPL',
 'Market Features': {'Market Return': 0.0250861159586972,
  'Market Volatility': 14.68000030517578,
  'Treasury Rate': 0.0549999997019767},
 'Surface': {'Log Moneyness': array([-0.31668849, -0.31668849, -0.30426597, ...,  0.63882295,
          0.6483924 ,  0.6483924 ]),
  'Time to Maturity': array([0.00793651, 0.00793651, 0.00793651, ..., 2.95634921, 2.95634921,
         2.95634921]),
  'Implied Volatility': array([0.3726, 0.6095, 0.3726, ..., 0.3387, 0.3342, 0.3389])}}

In [6]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import numpy as np

class IVSurfaceDataset(Dataset):
    def __init__(
        self, 
        data, 
        proportion, 
        random_state=0
    ):
        self.data = data
        self.proportion = proportion
        self.random_state = random_state

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        surface_data = self.data[idx]
        
        # Extract the surface coordinates and volatilities
        points_coordinates = np.stack([
            surface_data['Surface']['Log Moneyness'], 
            surface_data['Surface']['Time to Maturity']
        ], axis=1)
        points_volatilities = surface_data['Surface']['Implied Volatility']

        # Perform clustering
        n_clusters = int(np.ceil(1 / self.proportion))
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('kmeans', KMeans(n_clusters=n_clusters, random_state=self.random_state, n_init='auto'))
        ])
        labels = pipeline.fit_predict(points_coordinates)

        rng = np.random.default_rng(self.random_state)
        cluster_indices = np.where(labels == rng.integers(n_clusters))[0]
        num_to_mask = int(np.ceil(len(cluster_indices) * self.proportion))
        masked_indices = rng.choice(cluster_indices, size=num_to_mask, replace=False)
        
        unmasked_indices = np.setdiff1d(cluster_indices, masked_indices)


        data_item = {
            'Datetime': surface_data['Datetime'],
            'Symbol': surface_data['Symbol'],
            'Market Features': {
                'Market Return': torch.tensor(surface_data['Market Features']['Market Return'], dtype=torch.float32),
                'Market Volatility': torch.tensor(surface_data['Market Features']['Market Volatility'], dtype=torch.float32),
                'Treasury Rate': torch.tensor(surface_data['Market Features']['Treasury Rate'], dtype=torch.float32),
            },
            'Input Surface': {
                'Log Moneyness': torch.tensor(points_coordinates[unmasked_indices, 0], dtype=torch.float32),
                'Time to Maturity': torch.tensor(points_coordinates[unmasked_indices, 1], dtype=torch.float32),
                'Implied Volatility': torch.tensor(points_volatilities[unmasked_indices], dtype=torch.float32)
            },
            'Query Points': {
                'Log Moneyness': torch.tensor(points_coordinates[masked_indices, 0], dtype=torch.float32),
                'Time to Maturity': torch.tensor(points_coordinates[masked_indices, 1], dtype=torch.float32),
                'Implied Volatility': torch.tensor(points_volatilities[masked_indices], dtype=torch.float32)
            }
        }

        return data_item

    @staticmethod
    def collate_fn(batch):
        batched_data = {
            'Datetime': [item['Datetime'] for item in batch],
            'Symbol': [item['Symbol'] for item in batch],
            'Market Features': {
                'Market Return': default_collate([item['Market Features']['Market Return'] for item in batch]),
                'Market Volatility': default_collate([item['Market Features']['Market Volatility'] for item in batch]),
                'Treasury Rate': default_collate([item['Market Features']['Treasury Rate'] for item in batch]),
            },
            'Input Surface': {
                'Log Moneyness': [item['Input Surface']['Log Moneyness'].clone().detach() for item in batch],
                'Time to Maturity': [item['Input Surface']['Time to Maturity'].clone().detach() for item in batch],
                'Implied Volatility': [item['Input Surface']['Implied Volatility'].clone().detach() for item in batch],
            },
            'Query Points': {
                'Log Moneyness': [item['Query Points']['Log Moneyness'].clone().detach().requires_grad_(True) for item in batch],
                'Time to Maturity': [item['Query Points']['Time to Maturity'].clone().detach().requires_grad_(True) for item in batch],
                'Implied Volatility': [item['Query Points']['Implied Volatility'].clone().detach() for item in batch],
            }
        }

        return batched_data


# Assuming surfaces is the output from the implied_volatility_surfaces function
proportion = 0.2  # example proportion
dataset = IVSurfaceDataset(surfaces, proportion)
data_loader = DataLoader(
    dataset, 
    batch_size=HYPERPARAMETERS['Input Preprocessing']['Batch Size'], 
    shuffle=True, 
    num_workers=0, 
    collate_fn=IVSurfaceDataset.collate_fn
)

# Fetch one batch from the DataLoader
batch = next(iter(data_loader))
batch

{'Datetime': [Timestamp('2013-06-10 00:00:00'),
  Timestamp('2013-01-28 00:00:00'),
  Timestamp('2013-05-20 00:00:00'),
  Timestamp('2013-03-07 00:00:00')],
 'Symbol': ['AAPL', 'AAPL', 'GOOGL', 'AAPL'],
 'Market Features': {'Market Return': tensor([-0.0003, -0.0019, -0.0007,  0.0018]),
  'Market Volatility': tensor([15.4400, 13.5700, 13.0200, 13.0600]),
  'Treasury Rate': tensor([0.0400, 0.0600, 0.0350, 0.0900])},
 'Input Surface': {'Log Moneyness': [tensor([0.4524, 0.4597, 0.4668, 0.4739, 0.4739, 0.4880, 0.4950, 0.4950, 0.5019,
           0.5019, 0.5088, 0.5088, 0.5156, 0.5156, 0.5224, 0.5224, 0.5291, 0.5358,
           0.5358, 0.5425, 0.5491, 0.5556, 0.5556, 0.5621, 0.5621, 0.5686, 0.5686,
           0.5750, 0.5750, 0.5814, 0.5814, 0.5878, 0.5878, 0.5941, 0.6004, 0.6004,
           0.6066, 0.6066, 0.6128, 0.6128, 0.6189, 0.6251, 0.6251, 0.6311, 0.6372,
           0.6372, 0.6432, 0.6492, 0.6492, 0.6551, 0.6551, 0.6610, 0.6610, 0.6669,
           0.6669, 0.6727, 0.6727, 0.6785, 0.6785,

## Surface Embedding

### Components

In [7]:
import torch
import torch.nn as nn
from torch.utils.data._utils.collate import default_collate

class SurfaceBatchNorm(nn.Module):
    def __init__(
        self, 
        num_features=1, 
        momentum=0.1
    ):
        super(SurfaceBatchNorm, self).__init__()
        self.log_moneyness_bn = nn.BatchNorm1d(num_features, momentum=momentum)
        self.time_to_maturity_bn = nn.BatchNorm1d(num_features, momentum=momentum)
        self.market_return_bn = nn.BatchNorm1d(num_features, momentum=momentum)
        self.market_volatility_bn = nn.BatchNorm1d(num_features, momentum=momentum)
        self.treasury_rate_bn = nn.BatchNorm1d(num_features, momentum=momentum)

    def forward(self, batch):
        # Concatenate all tensors from the Input Surface into one tensor for each feature
        input_surface_log_moneyness = torch.cat([x for x in batch['Input Surface']['Log Moneyness']])
        input_surface_time_to_maturity = torch.cat([x for x in batch['Input Surface']['Time to Maturity']])

        # Concatenate Input Surface tensors with Query Points tensors
        total_log_moneyness = torch.cat([input_surface_log_moneyness] + [x for x in batch['Query Points']['Log Moneyness']])
        total_time_to_maturity = torch.cat([input_surface_time_to_maturity] + [x for x in batch['Query Points']['Time to Maturity']])

        # Normalize Log Moneyness and Time to Maturity
        norm_log_moneyness = self.log_moneyness_bn(total_log_moneyness.unsqueeze(1)).squeeze(1)
        norm_time_to_maturity = self.time_to_maturity_bn(total_time_to_maturity.unsqueeze(1)).squeeze(1)

        # Split the normalized results back to corresponding structures
        input_surface_sizes = [len(x) for x in batch['Input Surface']['Log Moneyness']]
        query_points_sizes = [len(x) for x in batch['Query Points']['Log Moneyness']]
        total_input_size = sum(input_surface_sizes)

        # Normalizing Market Features
        market_features = batch['Market Features']
        norm_market_return = self.market_return_bn(market_features['Market Return'].unsqueeze(1)).squeeze(1)
        norm_market_volatility = self.market_volatility_bn(market_features['Market Volatility'].unsqueeze(1)).squeeze(1)
        norm_treasury_rate = self.treasury_rate_bn(market_features['Treasury Rate'].unsqueeze(1)).squeeze(1)

        # Reconstructing the batch with normalized data
        output = {
            'Datetime': batch['Datetime'],
            'Symbol': batch['Symbol'],
            'Market Features': {
                'Market Return': norm_market_return,
                'Market Volatility': norm_market_volatility,
                'Treasury Rate': norm_treasury_rate
            },
            'Input Surface': {
                'Log Moneyness': list(torch.split(norm_log_moneyness[:total_input_size], input_surface_sizes)),
                'Time to Maturity': list(torch.split(norm_time_to_maturity[:total_input_size], input_surface_sizes)),
                'Implied Volatility': batch['Input Surface']['Implied Volatility']
            },
            'Query Points': {
                'Log Moneyness': list(torch.split(norm_log_moneyness[total_input_size:], query_points_sizes)),
                'Time to Maturity': list(torch.split(norm_time_to_maturity[total_input_size:], query_points_sizes)),
                'Implied Volatility': batch['Query Points']['Implied Volatility']
            }
        }

        # Ensure requires_grad is True for query point values
        for key in output['Query Points']:
            if key != 'Implied Volatility':  # We only set requires_grad for Log Moneyness and Time to Maturity
                for tensor in output['Query Points'][key]:
                    tensor.requires_grad_()

        return output

# Usage
surfacebatchnorm = SurfaceBatchNorm()
processed_batch = surfacebatchnorm(batch)
processed_batch

{'Datetime': [Timestamp('2013-06-10 00:00:00'),
  Timestamp('2013-01-28 00:00:00'),
  Timestamp('2013-05-20 00:00:00'),
  Timestamp('2013-03-07 00:00:00')],
 'Symbol': ['AAPL', 'AAPL', 'GOOGL', 'AAPL'],
 'Market Features': {'Market Return': tensor([-0.0216, -0.4603, -0.1269,  0.6087], grad_fn=<SqueezeBackward1>),
  'Market Volatility': tensor([ 1.6897, -0.2052, -0.7625, -0.7220], grad_fn=<SqueezeBackward1>),
  'Treasury Rate': tensor([-0.7439,  0.1717, -0.9728,  1.5450], grad_fn=<SqueezeBackward1>)},
 'Input Surface': {'Log Moneyness': [tensor([1.2476, 1.2676, 1.2873, 1.3070, 1.3070, 1.3458, 1.3650, 1.3650, 1.3841,
           1.3841, 1.4030, 1.4030, 1.4219, 1.4219, 1.4406, 1.4406, 1.4591, 1.4776,
           1.4776, 1.4959, 1.5141, 1.5322, 1.5322, 1.5501, 1.5501, 1.5680, 1.5680,
           1.5857, 1.5857, 1.6033, 1.6033, 1.6209, 1.6209, 1.6383, 1.6555, 1.6555,
           1.6727, 1.6727, 1.6898, 1.6898, 1.7068, 1.7236, 1.7236, 1.7404, 1.7571,
           1.7571, 1.7736, 1.7901, 1.7901, 1.

In [21]:
import torch
import torch.nn as nn
import numpy as np

class EllipticalRBFKernel(nn.Module):
    def __init__(
        self, 
        input_dim, 
        bandwidth
    ):
        super(EllipticalRBFKernel, self).__init__()
        self.bandwidth = bandwidth
        # Initialize the log of the scale vector to zero, which corresponds to scale factors of one
        self.log_scale = nn.Parameter(torch.zeros(input_dim))

    def forward(self, distances):
        # Convert log scale to actual scale values
        scale = torch.exp(self.log_scale)
        
        # Create a diagonal scale matrix
        scale_matrix = torch.diag(scale)

        # Calculate the scaled distances
        scaled_distances = distances @ scale_matrix @ distances.t()
        
        # Normalize by the trace of the scale matrix
        trace_scale_matrix = torch.trace(scale_matrix)
        normalized_distances = scaled_distances / trace_scale_matrix

        # Compute the RBF kernel output using the normalized distances
        kernel_values = torch.exp(-normalized_distances / (2 * self.bandwidth ** 2))

        return kernel_values

class SurfaceContinuousKernelEmbedding(nn.Module):
    def __init__(self, d_embedding):
        super(SurfaceContinuousKernelEmbedding, self).__init__()
        self.d_embedding = d_embedding

        # Initialize multiple RBF kernels, each with a different fixed bandwidth
        self.kernels = nn.ModuleList()
        for i in range(1, d_embedding + 1):
            bandwidth_value = torch.erfinv(torch.tensor(i / (d_embedding + 1))) * np.sqrt(2)
            self.kernels.append(EllipticalRBFKernel(bandwidth=bandwidth_value, input_dim=2))

        self.input_surface_layer_norm = nn.LayerNorm(d_embedding)
        self.query_points_layer_norm = nn.LayerNorm(d_embedding)

    def forward(self, input_surface_batch, query_points_batch):
        batch_size = len(input_surface_batch['Log Moneyness'])
        input_surface_embeddings = []
        query_points_embeddings = []

        for i in range(batch_size):
            # Extract the coordinates and implied volatilities for each surface in the batch
            surface_coords = torch.stack([
                input_surface_batch['Log Moneyness'][i], 
                input_surface_batch['Time to Maturity'][i]
            ], dim=-1)
            surface_ivs = input_surface_batch['Implied Volatility'][i]

            query_coords = torch.stack([
                query_points_batch['Log Moneyness'][i], 
                query_points_batch['Time to Maturity'][i]
            ], dim=-1)

            all_coords = torch.cat((surface_coords, query_coords), dim=0)

            # Compute the pairwise differences between all points and the input surface points
            point_differences = all_coords.unsqueeze(1) - surface_coords.unsqueeze(0)  # (n+m, n, 2)

            # Initialize the output embeddings for the current surface with d_embedding channels
            all_embedded = torch.zeros((all_coords.shape[0], self.d_embedding), dtype=torch.float32, device=surface_coords.device)

            for kernel_idx, kernel in enumerate(self.kernels):
                # Apply the RBF kernel to each distance vector using torch.vmap
                vmap_kernel = torch.vmap(kernel, in_dims=(0,))
                kernel_outputs = vmap_kernel(point_differences.view(-1, point_differences.shape[-1]))  # ((n+m) * n)
                kernel_outputs = kernel_outputs.view(all_coords.shape[0], surface_coords.shape[0])  # (n+m, n)

                # Compute the weighted sum of IVs based on the kernel outputs
                weighted_sum = (kernel_outputs * surface_ivs.unsqueeze(0)).sum(dim=1)
                normalization_factor = kernel_outputs.sum(dim=1)

                all_embedded[:, kernel_idx] = weighted_sum / normalization_factor

            # Split the embeddings into input surface and query points embeddings
            input_surface_embedded = all_embedded[:surface_coords.shape[0], :]
            query_points_embedded = all_embedded[surface_coords.shape[0]:, :]

            # Normalize the embedded surfaces
            input_surface_embedded = self.input_surface_layer_norm(input_surface_embedded)
            query_points_embedded = self.query_points_layer_norm(query_points_embedded)

            # Append the encoded surface for this input surface to the batch list
            input_surface_embeddings.append(input_surface_embedded)
            query_points_embeddings.append(query_points_embedded)

        # Keep all encoded surfaces as lists to handle variable lengths
        return {
            'Input Surface': input_surface_embeddings,
            'Query Points': query_points_embedded
        }


# Example of initializing and using this module
d_embedding = HYPERPARAMETERS['Input Embedding']['Surface Embedding']['Channels Dimension']  # Desired number of output channels

continuous_kernel_embedding = SurfaceContinuousKernelEmbedding(d_embedding=d_embedding)
embedded_batch = continuous_kernel_embedding(processed_batch['Input Surface'], processed_batch['Query Points'])
embedded_batch

{'Input Surface': [tensor([[ 1.2387,  0.6684,  0.2757,  ..., -0.4899, -0.6068, -0.6940],
          [ 1.1609,  0.6920,  0.3154,  ..., -0.4901, -0.6148, -0.7079],
          [ 1.0836,  0.7125,  0.3540,  ..., -0.4893, -0.6218, -0.7208],
          ...,
          [-0.5958, -0.7983, -0.3006,  ...,  0.3868,  0.4721,  0.5335],
          [-0.5958, -0.7983, -0.3006,  ...,  0.3868,  0.4721,  0.5335],
          [-0.5902, -0.7512, -0.2647,  ...,  0.3648,  0.4424,  0.4983]],
         grad_fn=<NativeLayerNormBackward0>),
  tensor([[ 2.2988,  0.7190,  0.0672,  ..., -0.6527, -0.7860, -0.8957],
          [ 2.0635,  0.8940,  0.3154,  ..., -0.7051, -0.9214, -1.1002],
          [ 2.0635,  0.8940,  0.3154,  ..., -0.7051, -0.9214, -1.1002],
          ...,
          [-1.3325, -1.1211, -0.7927,  ...,  0.7208,  1.1328,  1.4756],
          [-1.3237, -1.1212, -0.7984,  ...,  0.7209,  1.1348,  1.4789],
          [-1.3237, -1.1212, -0.7984,  ...,  0.7209,  1.1348,  1.4789]],
         grad_fn=<NativeLayerNormBackward